# Imputation

Version: 2024-10-9

*Imputation* is the process of replacing missing data with substituted values.

## A. Basic Setup

Let us begin by importing the data we need using `pandas`.

In [2]:
import pandas as pd

# Import data
gdp = pd.read_excel("../Data/hk-gdp.xlsx")
unemployment = pd.read_excel("../Data/unemployment.xlsx")

In [3]:
# gdp data
gdp.head()

,year,quarter,gdp
0,2010,1,422783
1,2010,2,412768
2,2010,3,456830
3,2010,4,483951
4,2011,1,463467


In [4]:
# Unemployment rate data
unemployment.head()

,year,end-month,unemployment-rate
0,2010,1,4.6
1,2010,2,4.4
2,2010,3,4.4
3,2010,4,4.6
4,2010,5,4.8


In order to merge the two sets of data, we need to generate `end-month` for `gdp`. We will also compute quarter-to-quarter GDP growth.

In [5]:
# Create end-month
gdp['end-month'] = gdp['quarter'] * 3

# Create gdp_growth
gdp['gdp_growth'] = gdp['gdp']/gdp['gdp'].shift(1) - 1

gdp.head()

,year,quarter,gdp,end-month,gdp_growth
0,2010,1,422783,3,NaN
1,2010,2,412768,6,-0.023688
2,2010,3,456830,9,0.106748
3,2010,4,483951,12,0.059368
4,2011,1,463467,3,-0.042327


Now let us merge the two datasets:

In [6]:
# Merge data
merged_data = unemployment.merge(gdp, how='left', on=['year','end-month'])
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_growth
0,2010,1,4.6,NaN,NaN,NaN
1,2010,2,4.4,NaN,NaN,NaN
2,2010,3,4.4,1.0,422783.0,NaN
3,2010,4,4.6,NaN,NaN,NaN
4,2010,5,4.8,NaN,NaN,NaN
5,2010,6,4.8,2.0,412768.0,-0.023688
6,2010,7,4.6,NaN,NaN,NaN
7,2010,8,4.6,NaN,NaN,NaN
8,2010,9,4.4,3.0,456830.0,0.106748
9,2010,10,4.2,NaN,NaN,NaN


## B. Pandas: Replace Missing Values with a Single Value

```python
DataFrame['new_column'] = DataFrame['existing_col'].fillna(DataFrame['existing_col'].ops())
```

For example, if we would like to replace missing GDP values with the mean of the same series:

In [7]:
merged_data['gdp_imputed'] = merged_data['gdp'].fillna(merged_data['gdp'].mean())
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_growth,gdp_imputed
0,2010,1,4.6,NaN,NaN,NaN,590507.906977
1,2010,2,4.4,NaN,NaN,NaN,590507.906977
2,2010,3,4.4,1.0,422783.0,NaN,422783.000000
3,2010,4,4.6,NaN,NaN,NaN,590507.906977
4,2010,5,4.8,NaN,NaN,NaN,590507.906977
5,2010,6,4.8,2.0,412768.0,-0.023688,412768.000000
6,2010,7,4.6,NaN,NaN,NaN,590507.906977
7,2010,8,4.6,NaN,NaN,NaN,590507.906977
8,2010,9,4.4,3.0,456830.0,0.106748,456830.000000
9,2010,10,4.2,NaN,NaN,NaN,590507.906977


If you prefer to replace the original column instead of generating a new one, you can add the option `inplace=True`:

```python
DataFrame['existing_col'].fillna(DataFrame['existing_col'].ops(), inplace=True)
```

## C. Pandas: Index and Interpolation

If you want to fill missing values using interpolation instead of a single value, you will have to make a decision on the format of the index, because this affects the types of interpolation pandas allows you to use.

First let us try using more than one column as the index. This is called `MultiIndex` in pandas:

In [8]:
merged_data.index = [merged_data['year'],merged_data['end-month']]
merged_data.head(12)

year  end-month  unemployment-rate  quarter       gdp  \
year end-month                                                          
2010 1          2010          1                4.6      NaN       NaN   
     2          2010          2                4.4      NaN       NaN   
     3          2010          3                4.4      1.0  422783.0   
     4          2010          4                4.6      NaN       NaN   
     5          2010          5                4.8      NaN       NaN   
     6          2010          6                4.8      2.0  412768.0   
     7          2010          7                4.6      NaN       NaN   
     8          2010          8                4.6      NaN       NaN   
     9          2010          9                4.4      3.0  456830.0   
     10         2010         10                4.2      NaN       NaN   
     11         2010         11                3.9      NaN       NaN   
     12         2010         12                3.7      4.0  483951.0   

                gdp_growth    gdp_imputed  
year end-month                             
2010 1                 NaN  590507.906977  
     2                 NaN  590507.906977  
     3                 NaN  422783.000000  
     4                 NaN  590507.906977  
     5                 NaN  590507.906977  
     6           -0.023688  412768.000000  
     7                 NaN  590507.906977  
     8                 NaN  590507.906977  
     9            0.106748  456830.000000  
     10                NaN  590507.906977  
     11                NaN  590507.906977  
     12           0.059368  483951.000000

The syntax for interpolating a column is: 

```python
DataFrame['new_column'] = DataFrame['existing_column'].interpolate(method='some_method')
```

`MultiIndex` only supports linear interpolation, which treats all observations as equally spaced:

In [9]:
merged_data['gdp_imputed'] = merged_data['gdp'].interpolate()
merged_data.head(12)

year  end-month  unemployment-rate  quarter       gdp  \
year end-month                                                          
2010 1          2010          1                4.6      NaN       NaN   
     2          2010          2                4.4      NaN       NaN   
     3          2010          3                4.4      1.0  422783.0   
     4          2010          4                4.6      NaN       NaN   
     5          2010          5                4.8      NaN       NaN   
     6          2010          6                4.8      2.0  412768.0   
     7          2010          7                4.6      NaN       NaN   
     8          2010          8                4.6      NaN       NaN   
     9          2010          9                4.4      3.0  456830.0   
     10         2010         10                4.2      NaN       NaN   
     11         2010         11                3.9      NaN       NaN   
     12         2010         12                3.7      4.0  483951.0   

                gdp_growth    gdp_imputed  
year end-month                             
2010 1                 NaN            NaN  
     2                 NaN            NaN  
     3                 NaN  422783.000000  
     4                 NaN  419444.666667  
     5                 NaN  416106.333333  
     6           -0.023688  412768.000000  
     7                 NaN  427455.333333  
     8                 NaN  442142.666667  
     9            0.106748  456830.000000  
     10                NaN  465870.333333  
     11                NaN  474910.666667  
     12           0.059368  483951.000000

Next we will try a single index. We will need to combine year and month into a single number:

In [11]:
merged_data.index = (merged_data['year'] - 2010)*12 + merged_data['end-month']
merged_data.head(24)

,year,end-month,unemployment-rate,quarter,gdp,gdp_growth,gdp_imputed
1,2010,1,4.6,NaN,NaN,NaN,NaN
2,2010,2,4.4,NaN,NaN,NaN,NaN
3,2010,3,4.4,1.0,422783.0,NaN,422783.000000
4,2010,4,4.6,NaN,NaN,NaN,419444.666667
5,2010,5,4.8,NaN,NaN,NaN,416106.333333
6,2010,6,4.8,2.0,412768.0,-0.023688,412768.000000
7,2010,7,4.6,NaN,NaN,NaN,427455.333333
8,2010,8,4.6,NaN,NaN,NaN,442142.666667
9,2010,9,4.4,3.0,456830.0,0.106748,456830.000000
10,2010,10,4.2,NaN,NaN,NaN,465870.333333


A single index allows for many more [interpolations methods](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate). The default interoplation method is `linear`, giving the same result as before:

In [12]:
merged_data['gdp_imputed'] = merged_data['gdp'].interpolate()
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_growth,gdp_imputed
1,2010,1,4.6,NaN,NaN,NaN,NaN
2,2010,2,4.4,NaN,NaN,NaN,NaN
3,2010,3,4.4,1.0,422783.0,NaN,422783.000000
4,2010,4,4.6,NaN,NaN,NaN,419444.666667
5,2010,5,4.8,NaN,NaN,NaN,416106.333333
6,2010,6,4.8,2.0,412768.0,-0.023688,412768.000000
7,2010,7,4.6,NaN,NaN,NaN,427455.333333
8,2010,8,4.6,NaN,NaN,NaN,442142.666667
9,2010,9,4.4,3.0,456830.0,0.106748,456830.000000
10,2010,10,4.2,NaN,NaN,NaN,465870.333333


Another possibility is to use either `.ffill()` or `.bfill()`, which use the closest non-missing value from the past or the future respectively:

In [13]:
merged_data['gdp_imputed'] = merged_data['gdp'].ffill() #bfill for back fill
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_growth,gdp_imputed
1,2010,1,4.6,NaN,NaN,NaN,NaN
2,2010,2,4.4,NaN,NaN,NaN,NaN
3,2010,3,4.4,1.0,422783.0,NaN,422783.0
4,2010,4,4.6,NaN,NaN,NaN,422783.0
5,2010,5,4.8,NaN,NaN,NaN,422783.0
6,2010,6,4.8,2.0,412768.0,-0.023688,412768.0
7,2010,7,4.6,NaN,NaN,NaN,412768.0
8,2010,8,4.6,NaN,NaN,NaN,412768.0
9,2010,9,4.4,3.0,456830.0,0.106748,456830.0
10,2010,10,4.2,NaN,NaN,NaN,456830.0


## D. Scikit-learn Imputers

You can also use scikit-learn's imputation classes. The `SimpleImputer` class replaces missing values with a single value, while the `IterativeImputer` replaces missing values by the prediction of a model fitted on non-missing values.

Let us first try the `SimpleImputer`:

In [94]:
import numpy as np
from sklearn.impute import SimpleImputer

# Replace missing values with the mean of the series
imp = SimpleImputer(strategy='mean')
X = imp.fit_transform(merged_data[['gdp']])
merged_data['gdp_imputed'] = X
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_imputed
0,2010,1,4.6,NaN,590507.906893,590507.906893
1,2010,2,4.4,NaN,590507.906908,590507.906908
2,2010,3,4.4,1.0,422783.000000,422783.000000
3,2010,4,4.6,NaN,590507.906893,590507.906893
4,2010,5,4.8,NaN,590507.906878,590507.906878
5,2010,6,4.8,2.0,412768.000000,412768.000000
6,2010,7,4.6,NaN,590507.906893,590507.906893
7,2010,8,4.6,NaN,590507.906893,590507.906893
8,2010,9,4.4,3.0,456830.000000,456830.000000
9,2010,10,4.2,NaN,590507.906923,590507.906923


In [93]:
# Replace missing values with the most frequent value of the series
imp = SimpleImputer(strategy='most_frequent')
merged_data['gdp_imputed'] = imp.fit_transform(merged_data[['gdp']])
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_imputed
0,2010,1,4.6,NaN,590507.906893,590507.906893
1,2010,2,4.4,NaN,590507.906908,590507.906908
2,2010,3,4.4,1.0,422783.000000,422783.000000
3,2010,4,4.6,NaN,590507.906893,590507.906893
4,2010,5,4.8,NaN,590507.906878,590507.906878
5,2010,6,4.8,2.0,412768.000000,412768.000000
6,2010,7,4.6,NaN,590507.906893,590507.906893
7,2010,8,4.6,NaN,590507.906893,590507.906893
8,2010,9,4.4,3.0,456830.000000,456830.000000
9,2010,10,4.2,NaN,590507.906923,590507.906923


With `IterativeImputer`, you can choose a model to predict the missing values. The default is a Bayesian Ridge Regression, which is similar to the usual Ridge Regression but with the strength of regularization estimated from data. To predict the missing value of a variable, the model will use all other variables you provide. 

Since it does not make sense to predict the absolute level of GDP with unemployment rate, we will predict GDP growth instead.

In [110]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer()
X = imp.fit_transform(merged_data[['unemployment-rate','gdp_growth']])
merged_data['ur_imputed'] = X[:,0]
merged_data['gdpg_imputed'] = X[:,1]
merged_data.head(12)

,year,end-month,unemployment-rate,quarter,gdp,gdp_growth,ur_imputed,gdpg_imputed
0,2010,1,4.6,NaN,NaN,NaN,4.6,0.014928
1,2010,2,4.4,NaN,NaN,NaN,4.4,0.014782
2,2010,3,4.4,1.0,422783.0,NaN,4.4,0.014782
3,2010,4,4.6,NaN,NaN,NaN,4.6,0.014928
4,2010,5,4.8,NaN,NaN,NaN,4.8,0.015074
5,2010,6,4.8,2.0,412768.0,-0.023688,4.8,-0.023688
6,2010,7,4.6,NaN,NaN,NaN,4.6,0.014928
7,2010,8,4.6,NaN,NaN,NaN,4.6,0.014928
8,2010,9,4.4,3.0,456830.0,0.106748,4.4,0.106748
9,2010,10,4.2,NaN,NaN,NaN,4.2,0.014636
